## Predicting customer spending 

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lreg').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('data/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.show(1)

+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+
|               Email|             Address|Avatar|Avg Session Length|      Time on App|  Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|Violet| 34.49726772511229|12.65565114916675|39.57766801952616|  4.0826206329529615|  587.9510539684005|
+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+
only showing top 1 row



In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership', 'Yearly Amount Spent'],
                            outputCol='features')

In [10]:
output = assembler.transform(data)

In [11]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [12]:
final_data = output.select('features', 'Yearly Amount Spent')

In [13]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [14]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [15]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                354|
|   mean|  499.7153535673652|
| stddev|  77.07373152664636|
|    min|  275.9184206503857|
|    max|  765.5184619388373|
+-------+-------------------+



In [16]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [17]:
lr_model = lr.fit(train_data)

In [18]:
test_results = lr_model.evaluate(test_data)

In [19]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-4.32009983342140...|
|8.355982572538778...|
|2.216893335571512...|
|-2.78532752417959...|
|3.240074875066057...|
|3.751665644813329...|
|-4.83169060316868...|
|4.490630090003833...|
|8.526512829121202...|
|1.278976924368180...|
|3.069544618483633...|
|5.798028723802417...|
|8.355982572538778...|
|-1.13686837721616...|
|9.094947017729282...|
|3.240074875066057...|
|4.604316927725449...|
|8.412825991399586...|
|-2.50111042987555...|
|-3.52429196937009...|
+--------------------+
only showing top 20 rows



In [20]:
test_results.rootMeanSquaredError

4.8151819311155636e-12

In [21]:
test_results.r2

1.0

In [22]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [23]:
unlabeled_data = test_data.select('features')

In [24]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.8162006488763...|
|[30.8364326747734...|
|[30.9716756438877...|
|[31.0613251567161...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.5261978982398...|
|[31.5702008293202...|
|[31.6739155032749...|
|[31.7242025238451...|
|[31.8093003166791...|
|[31.8164283341993...|
|[31.8293464559211...|
|[31.8627411090001...|
|[31.8745516945853...|
|[31.8854062999117...|
|[31.9048571310136...|
|[31.9480174211613...|
|[31.9549038566348...|
+--------------------+
only showing top 20 rows



In [25]:
predicitons = lr_model.transform(unlabeled_data)

In [26]:
predicitons.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 408.6403510726318|
|[30.8162006488763...|266.08634094846064|
|[30.8364326747734...| 467.5019004269874|
|[30.9716756438877...| 494.6386097568955|
|[31.0613251567161...|487.55545805789836|
|[31.1280900496166...| 557.2526867470509|
|[31.1695067987115...|427.35653080229764|
|[31.5261978982398...| 409.0945261923333|
|[31.5702008293202...| 545.9454921413964|
|[31.6739155032749...| 475.7250679098684|
|[31.7242025238451...| 503.3878872879574|
|[31.8093003166791...| 536.7718993628354|
|[31.8164283341993...|  501.122491503648|
|[31.8293464559211...| 385.1523379879751|
|[31.8627411090001...| 556.2981411740458|
|[31.8745516945853...|392.28524424626426|
|[31.8854062999117...| 390.1032729724709|
|[31.9048571310136...| 473.9498574228077|
|[31.9480174211613...| 461.9208768929003|
|[31.9549038566348...| 439.9978799399305|
+--------------------+------------